In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from  tensorflow.keras import models, optimizers, regularizers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt 

In [3]:
!pip install flask-ngrok

In [4]:
import PIL.Image as Image
import io
import base64
#from data import byte_data

In [5]:
from google.colab import drive
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
drive.mount('/content/drive')

Mounted at /content/drive



Leer ubicaciones de las imagenes y del modelo

In [6]:
path_img='/content/drive/MyDrive/ciudadlimpia/uploadImages/'
#path_model='/content/drive/MyDrive/ciudadlimpia/model/my_model2.h5'
path_model='/content/drive/MyDrive/ciudadlimpia/model/my_modelincp5.h5'
path_results='/content/drive/MyDrive/ciudadlimpia/resultsImages/'

In [7]:
# y para cara cargarlo utilizamos el mismo metodo 
model_complete4 = tf.keras.models.load_model(path_model)

In [8]:
#pasarle la ruta de la imagen del drive para llevarlo a un objeto tensor
#para poder realizar la prediccion
#con el metodo predict del modelo
def imageToTensorImagen(img_path ,target_size_var, axis):
    img = keras.preprocessing.image.load_img(img_path, target_size = target_size_var )
    ggArregloNumpyImagen = keras.preprocessing.image.img_to_array(img)
    ggTensorImgenPrediccion=tf.expand_dims(ggArregloNumpyImagen,axis)
    return(ggTensorImgenPrediccion/255)

In [9]:
def model_predict(model, imagePath):
    # test_datagen = ImageDataGenerator(rescale=1./255,validation_split= 0.2)
    test_dir= imagePath

    test_img_resize = imageToTensorImagen(imagePath,(150, 150),0)
    predict=model.predict(test_img_resize)
    predict=transfor_predict(predict,0.50)
    return(str(predict))


In [10]:
def transfor_predict(predict,umbral):
  x=0
  for y in predict:
    if y > umbral :
      x=1
    else:
      x=0
  return(x) 

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify, render_template,request
app = Flask(__name__)
run_with_ngrok(app) #starts ngrok when the app is running

@app.route('/', methods=['GET','POST'])
def home():
    if request.method=='GET':

        resultado = {"result':'voila"}
        return resultado
    if request.method=='POST':
        results_paths = path_results + 'resultados.csv'
        imageID = request.form.get('filename')
        Byte_Im = request.form.get('file')
        fechacompleta = request.form.get('fecha')
        latitud = request.form.get('latitud')
        longitud = request.form.get('longitud')

        imagenpath = path_img + imageID
        #*************  decodificar para pasarlo a imagen a la carpeta
        image2bytes = Byte_Im.encode('utf-8')
        #image2bytes = Byte_Im
        with open(imagenpath,'wb') as file_to_save:
            decoded_image_data = base64.decodebytes(image2bytes)
            file_to_save.write(decoded_image_data)
        #************
        prediccion=model_predict(model_complete4,imagenpath)
        

        cadenainsert = fechacompleta + ',' + fechacompleta + ',' + latitud + ',' + longitud + ',' + prediccion + ',' + imagenpath

        # Open a file with access mode 'a'
        with open(results_paths,'a') as results_to_save:
            # Append 'hello' at the end of file
            results_to_save.write(cadenainsert+"\n")
            
        #************
        resultado = {"result":prediccion}
        return resultado
    else:
      resultado = {"result":"NOPOST"}
      return resultado 

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c3b3-34-80-251-100.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Feb/2022 22:22:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 22:23:06] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 22:29:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 22:32:09] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 22:32:47] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 22:39:40] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 22:45:35] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 22:57:02] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 22:57:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 22:59:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 23:02:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 23:03:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 23:05:56] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2022 23:06:11] "POST / HTTP/1.1" 200 -
